In [1]:
import pandas as pd 
import numpy as np
from Tools import TrateNan, plot_predict

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
output_notebook()

from sklearn.preprocessing import Imputer, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from xgboost import XGBRegressor



pd.set_option("display.max_columns",600)
pd.options.display.float_format = '{:.1f}'.format

Loading BokehJS ...

In [2]:
df=pd.read_csv("../../../Documentos/Data/Examen/train.csv")

In [3]:
df.shape

(526, 547)

In [4]:
obj=df.target

In [5]:
tratenan=TrateNan(df)

In [6]:
del df
df=tratenan.df_to_var(tol_car=10,tol_nan=.2).copy()

Tools.py:234: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  "Cardinalidad":[len(ser.value_counts().index)]})])


In [7]:
Incremento=lambda l: sum([int(x<y) for x, y in zip(l,l[1:])])
Delta=lambda l: np.mean([(x-y) for x, y in zip(l,l[1:])])

In [8]:
df["Incremento"]=df[[x for x in df.columns 
                if "ventas" in x]].apply(
                Incremento, axis=1)


df["Delta"]=df[[x for x in df.columns if "ventas" in x]].apply(
    Delta, axis=1)

In [9]:
df["target"]=obj.copy()

In [10]:
df.dropna(subset=["target"], inplace=True)
df.reset_index(drop=True, inplace=True)

In [11]:
imputer=Imputer(strategy='mean', axis=0)
imputer.fit(df[[x for x in df.columns if "v_" in x]])
df[[x for x in df.columns if "v_" in x]]=pd.DataFrame(imputer.transform(
                                         df[[x for x in df.columns if "v_" in x]]), 
                                        columns=[x for x in df.columns if "v_" in x])

In [12]:
X=df.iloc[:,:-1]
y=df[["target"]]

In [13]:
scx = MinMaxScaler()
scy = MinMaxScaler()
scx.fit(X)
scy.fit(y)
Xs = pd.DataFrame(scx.transform(X),columns=X.columns)
ys = pd.DataFrame(scy.transform(y))

In [14]:
Xt,Xv,yt,yv = train_test_split(Xs,ys,train_size=0.7, random_state=0)

/home/comdisde/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [39]:
model=ExtraTreesRegressor()
model.fit(Xt,yt)

/home/comdisde/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
          oob_score=False, random_state=None, verbose=0, warm_start=False)

In [40]:
print r2_score(y_pred=model.predict(Xt),y_true=yt)
print r2_score(y_pred=model.predict(Xv),y_true=yv)

1.0
0.324975095007933


In [44]:
param_grid = dict(n_estimators=range(100,200,10),
                  criterion = ['mse','mae'],
                  max_depth = range(10,20)
                 )

In [45]:
grid = RandomizedSearchCV(cv=3,verbose=False,
                          estimator=model,
                          n_jobs=-1,
                          param_distributions=param_grid,
                          scoring='r2')

In [46]:
grid.fit(Xs,ys)
model = grid.best_estimator_

/home/comdisde/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/comdisde/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/comdisde/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/comdisde/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_validation.py:458: Dat

/home/comdisde/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/comdisde/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/comdisde/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


In [47]:
print r2_score(y_pred=model.predict(Xt),y_true=yt)
print r2_score(y_pred=model.predict(Xv),y_true=yv)

0.9999892180965785
0.9999778805242349


In [48]:
p = figure(plot_width=980, plot_height=400, title="Train")
p.title.text_font_size = '20pt'


p.title.align="center"

p.line(y=yt[0],x=range(len(yt.values)),  line_width=3, color=(100,191,163), legend="Real")

p.circle(y=yt[0].where(yt[0]!=0),x=range(len(yt.values)), alpha=.6, size=9, color=(100,191,163))

y_pred=model.predict(Xt).reshape(-1, 1).reshape(1,315)[0]


p.line(y=y_pred,x=range(len(y_pred)),  line_width=2, color=(249, 139, 96), legend=u"Predicción")

p.circle(y=y_pred[y_pred!=0], x=np.where(y_pred!=0)[0], alpha=.4, size=9, color=(249, 139, 96))




show(p)



In [49]:
p = figure(plot_width=980, plot_height=400, title="Test-Validate")
p.title.text_font_size = '20pt'


p.title.align="center"

p.line(y=yv[0],x=range(len(yv.values)),  line_width=3, color=(100,191,163), legend="Real")

p.circle(y=yv[0].where(yv[0]!=0),x=range(len(yv.values)), alpha=.6, size=9, color=(100,191,163))

y_pred=model.predict(Xv).reshape(-1, 1).reshape(1,135)[0]


p.line(y=y_pred,x=range(len(y_pred)),  line_width=2, color=(249, 139, 96), legend=u"Predicción")

p.circle(y=y_pred[y_pred!=0], x=np.where(y_pred!=0)[0], alpha=.4, size=9, color=(249, 139, 96))




show(p)



In [50]:
df_v=pd.read_csv("../../../Documentos/Data/Examen/test.csv.csv")


In [51]:
df_v["Incremento"]=0
df_v["Delta"]=0
df_v=df_v[[x for x in df.columns if not(x in ["y^","target"])]].copy()

In [52]:
df_v[[x for x in df_v.columns if "v_" in x]]=pd.DataFrame(imputer.transform(
                                         df_v[[x for x in df_v.columns if "v_" in x]]), 
                                        columns=[x for x in df_v.columns if "v_" in x])

In [53]:
imputer_ventas=Imputer(strategy='mean', axis=1)
imputer_ventas.fit(df_v[[x for x in df_v.columns if "ventas" in x]])

df_v[[x for x in df_v.columns if "ventas" in x]]=pd.DataFrame(imputer_ventas.transform(
                                         df_v[[x for x in df_v.columns if "ventas" in x]]), 
                                        columns=[x for x in df_v.columns if "ventas" in x])


In [54]:
df_v["Incremento"]=df_v[[x for x in df_v.columns 
                if "ventas" in x]].apply(
                Incremento, axis=1)


df_v["Delta"]=df_v[[x for x in df_v.columns if "ventas" in x]].apply(
    Delta, axis=1)

In [55]:
df_v.isnull().sum()

ventas_mes_01_despues_de_lanzamiento    0
ventas_mes_02_despues_de_lanzamiento    0
ventas_mes_03_despues_de_lanzamiento    0
ventas_mes_04_despues_de_lanzamiento    0
ventas_mes_05_despues_de_lanzamiento    0
ventas_mes_06_despues_de_lanzamiento    0
v_1                                     0
v_2                                     0
v_3                                     0
v_4                                     0
v_15                                    0
c_2                                     0
c_3                                     0
c_4                                     0
c_5                                     0
c_6                                     0
c_7                                     0
c_8                                     0
c_15                                    0
c_384                                   0
c_460                                   0
c_461                                   0
c_490                                   0
Incremento                        

In [56]:
X_to_pred=pd.DataFrame(scx.transform(df_v), columns=df_v.columns)

In [57]:
plot_predict(y_real=[0], y_pred=model.predict(X_to_pred))